In [ ]:
# ===============================
# 🏨 ANALYSE DE LA DEMANDE HÔTELIÈRE
# ===============================
# Dataset : Hotel Booking Demand (Kaggle)
# Auteur : [Ton Nom]
# Date : Décembre 2025
# ===============================

# --- Importation des bibliothèques ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# --- Paramètres d’affichage ---
sns.set(style="whitegrid", palette="pastel")
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (10,5)

# --- Chargement du dataset ---
df = pd.read_csv("../data/hotel_bookings.csv")

# --- Aperçu des premières lignes ---
df.head()


In [ ]:
# Dimensions du dataset
print("Nombre de lignes :", df.shape[0])
print("Nombre de colonnes :", df.shape[1])

# Informations générales
df.info()

# Statistiques descriptives
df.describe()


In [ ]:
# Pourcentage de valeurs manquantes
missing = df.isnull().mean() * 100
missing = missing[missing > 0].sort_values(ascending=False)
print("Valeurs manquantes (%):")
print(missing)

# Suppression des doublons
print("Doublons avant nettoyage :", df.duplicated().sum())
df = df.drop_duplicates()
print("Doublons après nettoyage :", df.duplicated().sum())

# Remplissage simple des NaN enfants
df['children'] = df['children'].fillna(0)


In [ ]:
# Nombre de réservations par type d'hôtel
sns.countplot(data=df, x='hotel')
plt.title("Répartition des réservations par type d'hôtel")
plt.xlabel("Type d'hôtel")
plt.ylabel("Nombre de réservations")
plt.show()

# Taux d'annulation global
cancel_rate = df['is_canceled'].mean() * 100
print(f"Taux d'annulation global : {cancel_rate:.2f}%")

sns.countplot(data=df, x='is_canceled')
plt.title("Répartition des annulations")
plt.xlabel("0 = Réservée, 1 = Annulée")
plt.show()

# Taux d'annulation par type d'hôtel
cancel_by_hotel = df.groupby('hotel')['is_canceled'].mean() * 100
cancel_by_hotel.plot(kind='bar', color=['#66b3ff', '#ff9999'])
plt.title("Taux d'annulation (%) par type d'hôtel")
plt.ylabel("Taux (%)")
plt.show()

# Prix moyen (ADR) par type d'hôtel
sns.boxplot(data=df, x='hotel', y='adr')
plt.title("Distribution du prix moyen (ADR) selon le type d'hôtel")
plt.xlabel("Type d'hôtel")
plt.ylabel("Prix moyen par nuit (ADR)")
plt.show()

# Durée moyenne des séjours
df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
sns.boxplot(data=df, x='hotel', y='total_nights')
plt.title("Durée moyenne des séjours selon le type d'hôtel")
plt.xlabel("Type d'hôtel")
plt.ylabel("Nombre total de nuits")
plt.show()


In [ ]:
# Nombre de réservations par année
sns.countplot(data=df, x='arrival_date_year', hue='hotel')
plt.title("Nombre de réservations par année et par type d'hôtel")
plt.xlabel("Année d'arrivée")
plt.ylabel("Nombre de réservations")
plt.show()

# Saisonnalité (mois)
order_mois = ['January', 'February', 'March', 'April', 'May', 'June',
              'July', 'August', 'September', 'October', 'November', 'December']

sns.countplot(data=df, x='arrival_date_month', hue='hotel', order=order_mois)
plt.title("Réservations par mois et type d'hôtel")
plt.xticks(rotation=45)
plt.show()

# Lead time (délai entre réservation et arrivée)
sns.histplot(data=df, x='lead_time', bins=50, kde=True)
plt.title("Distribution du délai de réservation (Lead Time)")
plt.xlabel("Nombre de jours entre réservation et arrivée")
plt.show()


In [ ]:
# Nombre d'adultes / enfants / bébés
df[['adults', 'children', 'babies']].describe()

# Visualisation
df_melted = df.melt(value_vars=['adults', 'children', 'babies'],
                    var_name='Type', value_name='Nombre')
sns.boxplot(data=df_melted, x='Type', y='Nombre')
plt.title("Répartition du nombre d'adultes, d'enfants et de bébés")
plt.show()

# Types de clients
sns.countplot(data=df, x='customer_type', hue='hotel')
plt.title("Types de clients par type d'hôtel")
plt.xlabel("Type de client")
plt.ylabel("Nombre de réservations")
plt.show()

# Demandes spéciales
sns.boxplot(data=df, x='hotel', y='total_of_special_requests')
plt.title("Nombre de demandes spéciales par type d'hôtel")
plt.show()


In [ ]:
# Relation ADR - Annulation
sns.boxplot(data=df, x='is_canceled', y='adr')
plt.title("Relation entre prix moyen (ADR) et annulation")
plt.xlabel("Annulé (1) / Non annulé (0)")
plt.ylabel("Prix moyen (ADR)")
plt.show()

# Moyenne ADR par mois et type d'hôtel
adr_month = df.groupby(['arrival_date_month', 'hotel'])['adr'].mean().reset_index()
fig = px.line(adr_month, x='arrival_date_month', y='adr', color='hotel',
              title="Évolution du prix moyen (ADR) par mois et type d'hôtel",
              category_orders={'arrival_date_month': order_mois})
fig.show()


In [ ]:
# Distribution du prix moyen
fig1 = px.histogram(df, x="adr", color="hotel", nbins=50,
                    title="Distribution du prix moyen (ADR) par type d'hôtel",
                    marginal="box")
fig1.show()

# Annulations dans le temps
cancel_time = df.groupby(['arrival_date_year', 'hotel'])['is_canceled'].mean().reset_index()
fig2 = px.bar(cancel_time, x='arrival_date_year', y='is_canceled', color='hotel',
              title="Taux d'annulation par année et type d'hôtel")
fig2.update_yaxes(title_text="Taux d'annulation")
fig2.show()


In [ ]:
print("✅ Synthèse des principales observations :\n")
print("- Le City Hotel reçoit plus de réservations que le Resort Hotel.")
print("- Le taux d'annulation est plus élevé pour le City Hotel (~40–45%).")
print("- Le prix moyen (ADR) est plus élevé pour le Resort Hotel.")
print("- La demande est saisonnière : pics entre juin et août.")
print("- Les familles (adultes + enfants) réservent davantage le Resort Hotel.")
print("- Le délai de réservation (lead_time) peut dépasser 300 jours pour certains clients.")
print("\n⚠️ Limites du dataset :")
print("- Données entre 2015 et 2017 seulement.")
print("- Valeurs manquantes dans 'agent' et 'company'.")
print("- Aucune information sur les dépenses annexes (repas, services, etc.).")
